In [1]:
%reload_ext autoreload
%autoreload 2


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import seaborn as sns

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.size"] = 7
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True


In [3]:
import sys
sys.path.append('../')


In [4]:
import neptune


In [127]:
# https://docs.neptune.ai/api/project/#fetch_runs_table

tsdr_project = neptune.init_project(
    project="yuuk1/tsdr",
    mode="read-only",
)
tsdr_runs_table_df = tsdr_project.fetch_runs_table(state="inactive").to_pandas()


https://app.neptune.ai/yuuk1/tsdr/


In [128]:
loc_project = neptune.init_project(
    project="yuuk1/tsdr-localization",
    mode="read-only",
)
loc_runs_table_df = loc_project.fetch_runs_table(state="inactive").to_pandas()


https://app.neptune.ai/yuuk1/tsdr-localization/


In [129]:
def convert_to_dataset_name(x: pd.Series) -> str:
    match x["dataset/target_app"], x["dataset/metric_types/middlewares"], x["dataset/use_manually_selected_metrics"]:
        case "sock-shop", _, True:
            return "SS-small"
        case "sock-shop", False, False:
            return "SS-medium"
        case "sock-shop", True, False:
            return "SS-large"
        case "train-ticket", _, True:
            return "TT-small"
        case "train-ticket", False, False:
            return "TT-medium"
        case "train-ticket", True, False:
            return "TT-large"
        case _:
            raise ValueError(f"Unknown combination:")


In [130]:
tsdr_target_df = tsdr_runs_table_df[tsdr_runs_table_df["sys/failed"] == False][tsdr_runs_table_df["scores/cause_metrics/mcc_mean"] >= -1.0].sort_values(by="sys/creation_time").filter(
    regex="^sys/id|experiment_id|dataset/.+|parameters/.*|scores/.*",
).dropna(axis=1, how="all")
tsdr_target_df["dataset"] = tsdr_target_df.apply(convert_to_dataset_name, axis=1)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
display(tsdr_target_df)


,sys/id,dataset/dataset_id,dataset/metric_types/containers,dataset/metric_types/middlewares,dataset/metric_types/nodes,dataset/metric_types/services,dataset/sampling_scale_factor,dataset/target_app,dataset/time_range/end,dataset/time_range/start,...,scores/mandatory_tp,scores/num_series/containers,scores/num_series/middlewares,scores/num_series/services,scores/num_series/total,scores/reduction_rate_max,scores/reduction_rate_mean,scores/reduction_rate_min,scores/tp,dataset
163,TSDR-1656,9n6mf,True,False,False,True,1.0,sock-shop,0.0,0.0,...,69.0,0/0/65,NaN,0/0/15,43/64/80,0.796875,0.318741,0.061538,75.0,SS-small
162,TSDR-1657,9n6mf,True,False,False,True,1.0,sock-shop,0.0,0.0,...,76.0,0/0/484,NaN,0/0/15,97/185/499,0.824176,0.476571,0.142132,80.0,SS-medium
161,TSDR-1658,9n6mf,True,True,False,True,1.0,sock-shop,0.0,0.0,...,77.0,0/0/484,0/0/6294,0/0/15,569/1347/6793,0.784254,0.579265,0.350767,80.0,SS-large
160,TSDR-1659,9n6mf,True,False,False,True,1.0,sock-shop,0.0,0.0,...,70.0,0/0/65,NaN,0/0/15,42/64/80,0.796875,0.344285,0.061538,77.0,SS-small
159,TSDR-1660,9n6mf,True,False,False,True,1.0,sock-shop,0.0,0.0,...,76.0,0/0/484,NaN,0/0/15,94/185/499,0.835165,0.491847,0.172589,80.0,SS-medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,TSDR-1868,m9dgg,True,False,False,True,1.0,train-ticket,0.0,0.0,...,38.0,0/0/350,NaN,0/0/57,149/329/408,0.715026,0.546827,0.395833,41.0,TT-small
3,TSDR-1869,m9dgg,True,False,False,True,1.0,train-ticket,0.0,0.0,...,38.0,0/0/350,NaN,0/0/57,150/329/408,0.721831,0.546104,0.399306,41.0,TT-small
2,TSDR-1870,m9dgg,True,False,False,True,1.0,train-ticket,0.0,0.0,...,39.0,0/0/350,NaN,0/0/57,151/329/408,0.721831,0.542816,0.385417,42.0,TT-small
1,TSDR-1871,m9dgg,True,False,False,True,1.0,train-ticket,0.0,0.0,...,39.0,0/0/350,NaN,0/0/57,151/329/408,0.721831,0.542680,0.388889,42.0,TT-small


In [131]:
loc_target_df = loc_runs_table_df[loc_runs_table_df["sys/failed"] == False][loc_runs_table_df["experiment_id"] >= "20231003221628"].sort_values(by="sys/creation_time").filter(
    regex="^sys/id|experiment_id|dataset/.+|parameters/.*|scores/.*",
).dropna(axis=1, how="all")
loc_target_df["dataset"] = loc_target_df.apply(convert_to_dataset_name, axis=1)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
display(loc_target_df)


/tmp/ipykernel_4016986/674267130.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  loc_target_df = loc_runs_table_df[loc_runs_table_df["sys/failed"] == False][loc_runs_table_df["experiment_id"] >= "20231003221628"].sort_values(by="sys/creation_time").filter(


,sys/id,dataset/dataset_id,dataset/metric_types/containers,dataset/metric_types/middlewares,dataset/metric_types/nodes,dataset/metric_types/services,dataset/sampling_scale_factor,dataset/target_app,dataset/use_manually_selected_metrics,experiment_id,...,scores/service/AVG_2,scores/service/AVG_3,scores/service/AVG_4,scores/service/AVG_5,scores/service/AVG_6,scores/service/AVG_7,scores/service/AVG_8,scores/service/AVG_9,scores/service/num_cases,dataset
144,TSDRLOC-1952,m9dgg,True,False,False,True,1.0,train-ticket,True,20231003221628,...,0.352941,0.300654,0.269608,0.243922,0.224510,0.210444,0.197925,0.186827,17.0,TT-small
143,TSDRLOC-1953,m9dgg,True,False,False,True,1.0,train-ticket,False,20231003224108,...,0.328125,0.302083,0.277344,0.254375,0.236285,0.222938,0.210696,0.199631,16.0,TT-medium
142,TSDRLOC-1956,9n6mf,True,True,False,True,1.0,sock-shop,False,20231004171156,...,0.378125,0.352083,0.325000,0.299500,0.277014,0.257594,0.240824,0.226257,80.0,SS-large
141,TSDRLOC-1957,9n6mf,True,True,False,True,1.0,sock-shop,False,20231004171156,...,0.391447,0.366228,0.335526,0.307368,0.283187,0.262603,0.244991,0.229791,76.0,SS-large
140,TSDRLOC-1958,9n6mf,True,True,False,True,1.0,sock-shop,False,20231004171156,...,0.381250,0.352778,0.324740,0.299292,0.276840,0.257445,0.240694,0.226141,80.0,SS-large
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,TSDRLOC-2094,9n6mf,True,False,False,True,1.0,sock-shop,False,20231007180920,...,0.572368,0.486842,0.427632,0.382105,0.346199,0.317150,0.293131,0.272907,76.0,SS-medium
3,TSDRLOC-2095,9n6mf,True,False,False,True,1.0,sock-shop,False,20231007180920,...,0.588816,0.499269,0.436129,0.388377,0.351425,0.321630,0.297051,0.276391,76.0,SS-medium
2,TSDRLOC-2096,9n6mf,True,False,False,True,1.0,sock-shop,False,20231007180920,...,0.578947,0.489766,0.428180,0.381491,0.345322,0.316398,0.292473,0.272322,76.0,SS-medium
1,TSDRLOC-2097,9n6mf,True,False,False,True,1.0,sock-shop,False,20231007180920,...,0.569079,0.483187,0.424068,0.379254,0.343823,0.315114,0.291349,0.271323,76.0,SS-medium


In [132]:
tsdr_renamed_target_df = tsdr_target_df.copy()
tsdr_renamed_target_df.columns = tsdr_renamed_target_df.columns.str.replace("parameters/", "parameters/tsdr/")


In [133]:
merged_df = loc_target_df.merge(
    tsdr_renamed_target_df,
    on=loc_target_df.filter(regex="(parameters/tsdr/.*)|(dataset)").columns.tolist(),
    suffixes=("_loc", "_tsdr"),
    how="inner",
)
merged_df


,sys/id_loc,dataset/dataset_id,dataset/metric_types/containers,dataset/metric_types/middlewares,dataset/metric_types/nodes,dataset/metric_types/services,dataset/sampling_scale_factor,dataset/target_app,dataset/use_manually_selected_metrics,experiment_id_loc,...,scores/mandatory_fn,scores/mandatory_tp,scores/num_series/containers,scores/num_series/middlewares,scores/num_series/services,scores/num_series/total,scores/reduction_rate_max,scores/reduction_rate_mean,scores/reduction_rate_min,scores/tp
0,TSDRLOC-1952,m9dgg,True,False,False,True,1.0,train-ticket,True,20231003221628,...,4.0,38.0,0/0/350,NaN,0/0/57,149/329/408,0.715026,0.546709,0.395833,41.0
1,TSDRLOC-1952,m9dgg,True,False,False,True,1.0,train-ticket,True,20231003221628,...,4.0,38.0,0/0/350,NaN,0/0/57,149/329/408,0.715026,0.546827,0.395833,41.0
2,TSDRLOC-2031,m9dgg,True,False,False,True,1.0,train-ticket,True,20231006195224,...,4.0,38.0,0/0/350,NaN,0/0/57,149/329/408,0.715026,0.546709,0.395833,41.0
3,TSDRLOC-2031,m9dgg,True,False,False,True,1.0,train-ticket,True,20231006195224,...,4.0,38.0,0/0/350,NaN,0/0/57,149/329/408,0.715026,0.546827,0.395833,41.0
4,TSDRLOC-1953,m9dgg,True,False,False,True,1.0,train-ticket,False,20231003224108,...,0.0,42.0,0/0/2799,NaN,0/0/57,435/1250/2857,0.774303,0.652976,0.546569,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,TSDRLOC-2062,9n6mf,True,False,False,True,1.0,sock-shop,False,20231007180920,...,0.0,80.0,0/0/484,NaN,0/0/15,108/185/499,0.726257,0.418647,0.146465,80.0
172,TSDRLOC-2071,9n6mf,True,False,False,True,1.0,sock-shop,False,20231007180920,...,0.0,80.0,0/0/484,NaN,0/0/15,108/185/499,0.726257,0.418647,0.146465,80.0
173,TSDRLOC-2080,9n6mf,True,False,False,True,1.0,sock-shop,False,20231007180920,...,0.0,80.0,0/0/484,NaN,0/0/15,108/185/499,0.726257,0.418647,0.146465,80.0
174,TSDRLOC-2089,9n6mf,True,False,False,True,1.0,sock-shop,False,20231007180920,...,0.0,80.0,0/0/484,NaN,0/0/15,108/185/499,0.726257,0.418647,0.146465,80.0


In [134]:
merged_df.loc[:, merged_df.columns.str.startswith("scores/cause_metrics")].columns


Index(['scores/cause_metrics/bacc_mean', 'scores/cause_metrics/f1_score_mean',
       'scores/cause_metrics/fn_mean', 'scores/cause_metrics/fp_mean',
       'scores/cause_metrics/mcc_mean', 'scores/cause_metrics/num_found_mean',
       'scores/cause_metrics/num_mandatory_found_mean',
       'scores/cause_metrics/num_mandatory_total_mean',
       'scores/cause_metrics/num_total_mean',
       'scores/cause_metrics/precision_mean',
       'scores/cause_metrics/proportion_mandatory_mean',
       'scores/cause_metrics/proportion_mean',
       'scores/cause_metrics/recall_mandatory_mean',
       'scores/cause_metrics/recall_mean',
       'scores/cause_metrics/specificity_mean', 'scores/cause_metrics/tn_mean',
       'scores/cause_metrics/tp_mean'],
      dtype='object')

In [164]:
def detect_localization_method(row: pd.Series):
    pyrca_method = row.get("parameters/method", "")
    walk_method = row.get("parameters/walk_method", "")
    localization: str
    if pyrca_method == "epsilon_diagnosis":
        localization = "e-Diagnosis"
    elif type(row["parameters/use_rcd"]) != float and row["parameters/use_rcd"]:
        localization = "RCD"
    # elif type(row["parameters/use_causalrca"]) != float and row["parameters/use_causalrca"]:
    #     localization = "CausalRCA"
    elif pyrca_method == "pc" and walk_method == "rw-2":
        localization = "PC+RW-2"
    elif pyrca_method in ["ges", "fges"] and walk_method == "rw-2":
        localization = "GES+RW-2"
    elif pyrca_method == "lingam" and walk_method == "rw-2":
        localization = "LiNGAM+RW-2"
    elif pyrca_method == "call_graph" and walk_method == "rw-2":
        localization = "CG+RW-2"
    elif pyrca_method == "pc" and walk_method == "pagerank":
        localization = "PC+PageRank"
    elif pyrca_method in ["ges", "fges"] and walk_method == "pagerank":
        localization = "GES+PageRank"
    elif pyrca_method == "lingam" and walk_method == "pagerank":
        localization = "LiNGAM+PageRank"
    elif pyrca_method == "call_graph" and walk_method == "pagerank":
        localization = "CG+PageRank"
    elif pyrca_method == "pc" and walk_method == "ht":
        localization = "PC+HT"
    elif pyrca_method in ["ges", "fges"] and walk_method == "ht":
        localization = "GES+HT"
    elif pyrca_method == "lingam" and walk_method == "ht":
        localization = "LiNGAM+HT"
    elif pyrca_method == "call_graph" and walk_method == "ht":
        localization = "CG+HT"
    else:
        assert False, f"Unknown localization method: {row}"
    return localization

merged_df["localization_method"] = merged_df.apply(detect_localization_method, axis=1)
merged_df["localization_method"].unique()


array(['RCD', 'e-Diagnosis', 'CG+PageRank', 'PC+PageRank',
       'LiNGAM+PageRank', 'PC+HT', 'LiNGAM+HT'], dtype=object)

In [161]:

def detect_reduction_method(row: pd.Series):
    enable_uni: bool = row["parameters/tsdr/enable_unireducer"]
    enable_multi: bool = row["parameters/tsdr/enable_multireducer"]
    step1_model_name = row["parameters/tsdr/step1_model_name"]
    step1_method_name = row["parameters/tsdr/step1_method_name"]
    if type(step1_model_name) != float:
        uni_method = step1_model_name
    elif type(step1_method_name) != float:
        uni_method = step1_method_name
    else:
        uni_method = ""

    step2_clustering_method = row["parameters/tsdr/step2_clustering_method_name"]
    step2_dbscan_algorithm = row["parameters/tsdr/step2_dbscan_algorithm"]
    if type(step2_clustering_method) != float:
        multi_method = step2_clustering_method
    elif type(step2_dbscan_algorithm) != float:
        multi_method = step2_dbscan_algorithm
    else:
        multi_method = ""
    step2_dbscan_dist_type = row["parameters/tsdr/step2_dbscan_dist_type"]
    if type(step2_dbscan_dist_type) != float:
        multi_dist_type = step2_dbscan_dist_type
    else:
        multi_dist_type = ""

    reduction: str
    match (enable_uni, enable_multi, uni_method, multi_method, multi_dist_type):
        case (False, False, _, _, _):
            reduction = "None"
        case (True, True, "changepoint", "changepoint-kde", _):
            reduction = "MetricSifter"
        case (True, False, "zscore_nsigma", _, _):
            reduction = "NSigma"
        case (True, False, "birch_model", _, _):
            reduction = "BIRCH"
        case (True, False, "two_samp_test", _, _):
            reduction = "K-S test"
        case (True, False, "fluxinfer", _, _):
            reduction = "FluxInfer-AD"
        case (False, True, _, "hdbscan", "sbd"):
            reduction = "HDBS-SBD"
        case (False, True, _, "hdbscan", "pearsonr"):
            reduction = "HDBS-R"
        case (_, _, _, _, _):
            assert False, f"Unknown reduction method: {row}"

    return reduction

merged_df["reduction_method"] = merged_df.apply(detect_reduction_method, axis=1)
merged_df["reduction_method"].unique()


array(['MetricSifter', 'NSigma', 'BIRCH', 'K-S test', 'FluxInfer-AD',
       'HDBS-SBD', 'HDBS-R'], dtype=object)

In [187]:
loc_scores = ["scores/metric/AVG_5_mand", "scores/metric/AC_2_mand", "scores/metric/AC_5_mand"]
tsdr_scores = ["scores/cause_metrics/mcc_mean", "scores/cause_metrics/bacc_mean", "scores/cause_metrics/f1_score_mean", "scores/cause_metrics/recall_mean", "scores/cause_metrics/specificity_mean"]

loc_group_df = merged_df.groupby(["dataset", "reduction_method"], axis=0, as_index=True)

def correlation_fn(group: pd.Series, method="pearson"):
     # display(group[loc_scores], group[tsdr_scores])
     # print("print", group[tsdr_scores], group[loc_scores[-1]])
     z = group[tsdr_scores].corrwith(group[loc_scores[-1]], method=method, numeric_only=True, axis=0)
     return z

loc_group_df.apply(correlation_fn)


/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


,scores/cause_metrics/mcc_mean,scores/cause_metrics/bacc_mean,scores/cause_metrics/f1_score_mean,scores/cause_metrics/recall_mean,scores/cause_metrics/specificity_mean
dataset,,,,,
SS-large,0.980204,0.93627,0.943453,0.895370,-0.559831
SS-medium,-0.017747,-0.01842,-0.011607,-0.021671,0.022195
SS-small,0.089120,0.08893,0.086527,0.088930,0.033001
TT-medium,NaN,NaN,NaN,NaN,NaN
TT-small,-0.131994,-0.13078,-0.162726,-0.066405,-0.179511


In [137]:
# TODO: compute corr by each loc methods

for loc_perf in ["scores/metric/AVG_5_mand", "scores/metric/AC_2_mand", "scores/metric/AC_5_mand"]:
    display(
        merged_df[merged_df["parameters/use_rcd"] == True][["scores/cause_metrics/mcc_mean", "scores/cause_metrics/bacc_mean", "scores/cause_metrics/recall_mean", "scores/cause_metrics/specificity_mean"]].corrwith(merged_df[loc_perf], axis=0, numeric_only=True, method="spearman")
    )


scores/cause_metrics/mcc_mean            0.832058
scores/cause_metrics/bacc_mean          -0.002751
scores/cause_metrics/recall_mean         0.250250
scores/cause_metrics/specificity_mean   -0.301784
dtype: float64

scores/cause_metrics/mcc_mean            0.797798
scores/cause_metrics/bacc_mean          -0.052840
scores/cause_metrics/recall_mean         0.227191
scores/cause_metrics/specificity_mean   -0.305422
dtype: float64

scores/cause_metrics/mcc_mean            0.876059
scores/cause_metrics/bacc_mean           0.072644
scores/cause_metrics/recall_mean         0.345903
scores/cause_metrics/specificity_mean   -0.316945
dtype: float64